<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-del-projecto-equipo-47/blob/main/Reto_Entrega1_Equipo47.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Reto:  Entrega 1

## Materia: Ciencia y analítica de datos

Profa. María de la Paz Rico

Prof. Roberto Antonio Guevara González

## Nombres: Adriana Camarillo Durán, Laura Munoz Hernandez

## Matrícula: A01551139, A01167496

Fecha de entrega: 16/11/2022

# Limpieza de la base de datos


# Exploración de los datos

# K-means

## Resultados